# 1. Imports

In [1]:
import numpy as np
import pandas as pd
# scikit-learn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# pip install nltk
# pip install matplotlib
import nltk
import re
from nltk.stem import PorterStemmer
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import classification_report
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import random
import string

# Download necessary nltk data
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.corpus import wordnet

random.seed(42)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 2. Dataset Load

In [3]:
# Add the dataset the sample_data folder (on the left side)
# Define the column names based on the data format description
columns = ['title', 'from', 'genre', 'director', 'plot']

# Load the dataset
file_path = 'train.txt'
data = pd.read_csv(file_path, delimiter= "\t", names=columns) # Check the separator

# Inspect the first few rows
print(data.head())
print(data.shape)

# Pre-processing dataset
X_inicial = data['plot']  # Check if the appropriate collumn name is 'plot'

y = data['genre']      # Check if the appropriate collumn name is 'genre'
labels = np.unique(y).tolist()

                       title       from    genre       director  \
0               Ela Cheppanu     Telugu  romance         Ramana   
1  A Nightmare on Elm Street   American   horror   Samuel Bayer   
2            American Gothic   American   horror     John Hough   
3                       Gang  Bollywood    crime    Mazhar Khan   
4         Intimate Relations    British    drama  Charles Frank   

                                                plot  
0  Sekhar (Tarun) is a graduate from IIM and work...  
1  Kris Fowles (Katie Cassidy) goes to the Spring...  
2  Cynthia is traumatized by the death of her bab...  
3  Four friends, Gangu (Jackie Shroff), Abdul (Na...  
4  Crisis in a middle-class family when the son f...  
(8041, 5)


## 2.1 Dataset Load - retain only ADJ and ADV

In [ ]:
# Ensure you have the necessary NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Define the column names based on the data format description
columns = ['title', 'from', 'genre', 'director', 'plot']

# Load the dataset
file_path = 'train.txt'
data = pd.read_csv(file_path, delimiter="\t", names=columns)  # Ensure the correct separator

# Inspect the first few rows
print(data.head())
print(data.shape)

# Pre-processing dataset
X_inicial = data['plot']  # Ensure the appropriate column name is 'plot'
y = data['genre']         # Ensure the appropriate column name is 'genre'
labels = np.unique(y).tolist()

# Function to extract adjectives, adverbs, and keep punctuation like "!"
def extract_adj_adv_punct(text):
    # Tokenize text but keep punctuation
    tokens = nltk.word_tokenize(text)
    
    # Get POS tags
    pos_tags = nltk.pos_tag(tokens)
    
    # Filter adjectives (JJ), adverbs (RB), and exclamation marks (!)
    adj_adv_words = [word for word, tag in pos_tags if tag.startswith('JJ') or tag.startswith('RB')]
    
    # Manually add punctuation to be retained
    punct = re.findall(r'[!]', text)
    
    # Combine adjectives, adverbs, and punctuation
    return ' '.join(adj_adv_words + punct)

# Apply the function to extract adjectives, adverbs, and punctuation for each plot
X_inicial_adj_adv_punct = X_inicial.apply(extract_adj_adv_punct)

# Display the first few results
print(X_inicial_adj_adv_punct.head())

# Now X_inicial_adj_adv_punct contains only the adjectives, adverbs, and exclamation marks from each plot

## 2.2 Dataset Load - do sentiment analysis on each plot

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd

nltk.download('vader_lexicon')

# Inicializar o analisador de sentimentos
analyzer = SentimentIntensityAnalyzer()
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer

# Define the column names based on your data
columns = ['title', 'from', 'genre', 'director', 'plot']

# Load the dataset
file_path = 'train.txt'
data = pd.read_csv(file_path, delimiter="\t", names=columns)

# Check the first few rows and dataset shape
print(data.head())
print(data.shape)

# Ensure you have the correct columns for plot (X) and genre (y)
X_inicial = data['plot']
y = data['genre']
labels = np.unique(y).tolist()

# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Add a sentiment score (compound score) to each plot
data['sentiment'] = data['plot'].apply(lambda x: analyzer.polarity_scores(str(x))['compound'])

# Inspect the updated DataFrame with the new sentiment column
print(data.head())

### 2.2.1 Average sentiment by genre

In [17]:
# Group the data by genre and calculate the mean sentiment score for each genre
avg_sentiment_by_genre = data.groupby('genre')['sentiment'].mean().reset_index()

# Rename columns for clarity
avg_sentiment_by_genre.columns = ['genre', 'average_sentiment']

# Display the result
print(avg_sentiment_by_genre)

KeyError: 'Column not found: sentiment'

# 3. Pre-Processing

## 3.0 No Pre-Processing

In [ ]:
# If you don't want to apply any pre-processing: run this!!
X_process = X_inicial
y_process = y 

## 3.1 Remove Punctuation

In [ ]:
# if any pre-processing came first: run this!!!
X_inicial = X_process
y = y_process

In [5]:
def remove_punctuation_from_texts(X):
    # Translation table to remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    
    # Remove punctuation from each text in X
    X_cleaned = [text.translate(translator) for text in X]
    
    return X_cleaned

X_process = remove_punctuation_from_texts(X_inicial)
y_process = y
print(X_process[:10])

['Sekhar Tarun is a graduate from IIM and works as a business manager in a firm in Hyderabad He meets Amar Varma Siva Balaji in a business party who runs a company named Varma Industries in Germany When the party is over Sekhar accidentally kills Amar with his car Sekhars friend Sunil Sunil saves him by erasing the evidence but Sekhar feels guilty even after six months He goes to Germany along with Sunil to visit Amars family They find that their business is in a bad condition and want to help them by reviving it with his business skills Priya Shriya who was engaged to Amar earlier is trying her level best to run the business but helpless to run it smoothly Sekhar introduces himself to Priya and offer his help to revive the business The remaining story is about how Sekhar helps the business grow and win Priyas heart in that process', 'Kris Fowles Katie Cassidy goes to the Springwood Diner to meet with her exboyfriend Dean Russell Kellan Lutz who falls asleep at the table and meets a ma

## 3.2 Lowercasing

In [7]:
# if any preprocessing came first: run this!!!
X_inicial = X_process
y = y_process

In [9]:
def lowercase_texts(X):
    # Convert each text in X to lowercase
    X_lowercased = [text.lower() for text in X]
    
    return X_lowercased

X_process = lowercase_texts(X_inicial)
y_process = y
print(X_process[:10])

['sekhar tarun is a graduate from iim and works as a business manager in a firm in hyderabad he meets amar varma siva balaji in a business party who runs a company named varma industries in germany when the party is over sekhar accidentally kills amar with his car sekhars friend sunil sunil saves him by erasing the evidence but sekhar feels guilty even after six months he goes to germany along with sunil to visit amars family they find that their business is in a bad condition and want to help them by reviving it with his business skills priya shriya who was engaged to amar earlier is trying her level best to run the business but helpless to run it smoothly sekhar introduces himself to priya and offer his help to revive the business the remaining story is about how sekhar helps the business grow and win priyas heart in that process', 'kris fowles katie cassidy goes to the springwood diner to meet with her exboyfriend dean russell kellan lutz who falls asleep at the table and meets a ma

## 3.3 Remove Stop Words

In [11]:
# if any preprocessing came first: run this!!!
X_inicial = X_process
y = y_process

In [13]:
def remove_stopwords_from_texts(X, stop_words=None):
    if stop_words is None:
        print(stopwords.words('english'))
        stop_words = set(stopwords.words('english'))  # Load default NLTK stopwords
    
    # Remove stop words from each text in X
    X_cleaned = [' '.join([word for word in text.split() if word.lower() not in stop_words]) for text in X]
    
    return X_cleaned

Stop_words = None
#Stop_words=["the","is","and"]
X_process = remove_stopwords_from_texts(X_inicial, Stop_words)
y_process = y
print(X_process[:10])
print(f"Size of the data set: {len(y_process)}")

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## 3.4 Stemming

In [ ]:
# if any preprocessing came first: run this!!!
X_inicial = X_process
y = y_process

In [ ]:
# Pre-processing elements
porter_stemmer=PorterStemmer()
def processSentence(s):
    words=re.split("\\s+",s)
    stemmed_words=[porter_stemmer.stem(word=w) for w in words]
    return ' '.join(stemmed_words)


X_process = [processSentence(s) for s in X_inicial]
y_process = y
print(X_process[:10])
print(f"Size of the data set: {len(y_process)}")

## If you imported the data in 2.2 (sentiment analysis) - run this after pre-processing

In [ ]:
X_process = pd.DataFrame({
    'preprocessed_plot': X_process,  # The pre-processed text
    'sentiment': data['sentiment']  # Sentiment column
})

# 4. Train/Test Split

In [15]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_process, y_process, test_size=0.2, random_state=42)

# This is used to check the original movie plot (before vectorization) - to analyse the model's responses
x_test_raw = X_test

## 3.5 Balancing

In [ ]:
# To check how balanced is the dataset (oops)
y_train.value_counts().plot(kind = 'bar')


# Function to replace a word in a sentence with a synonym
'''The synonym_replacement function replaces random words in a sentence with their synonyms (if available in WordNet). 
This changes the text slightly but keeps the meaning intact.'''
def synonym_replacement(sentence, n=1):
    words = sentence.split()
    new_words = words.copy()
    # checks if a word has synonyms by looking up WordNet, a lexical database of English.
    random_word_list = list(set([word for word in words if wordnet.synsets(word)]))
    random.shuffle(random_word_list)
    # choses a random number of words to be replaced by their synonyms
    num_replacements = random.randint(n, len(random_word_list)) 
    for random_word in random_word_list[:num_replacements]:
        synonyms = wordnet.synsets(random_word)
        if synonyms:
            synonym = synonyms[0].lemmas()[0].name()  # Choose the first synonym
            new_words = [synonym if word == random_word else word for word in new_words]
    
    return ' '.join(new_words)

# Balancing the dataset using augmentation
def balance_dataset(X, y, labels):
    # Create a dataframe from X and y
    df = pd.DataFrame({'plot': X, 'genre': y})
    
    # Count the instances in each class
    class_counts = y.value_counts().to_dict()
    max_count = max(class_counts.values())  # Class with the most examples
    print(f"Initial class distribution: {class_counts}")
    
    # Augment underrepresented classes
    for label in labels:
        label_count = class_counts[label]
        if label_count < max_count:
            gap = max_count - label_count
            samples_to_augment = df[df['genre'] == label]['plot'].tolist()
            
            # Generate synthetic examples using synonym replacement and paraphrasing
            new_samples = []
            for i in range(gap):
                original_sample = random.choice(samples_to_augment)
                print(original_sample)
                augmented_sample = synonym_replacement(original_sample)
                print(augmented_sample)
                new_samples.append([augmented_sample, label])
            
            # Add new samples to the dataframe
            new_df = pd.DataFrame(new_samples, columns=['plot', 'genre'])
            df = pd.concat([df, new_df])
    
    print(f"Balanced class distribution: {df['genre'].value_counts().to_dict()}")
    print(f"Size of the data set: {df.shape}")
    return df['plot'], df['genre']

# Assuming X_inicial and y are defined as follows:
# X_inicial = data['plot']  # Movie plots
# y = data['genre']  # Movie genres
# labels = np.unique(y).tolist()

# Call the function to balance the dataset
X_train, y_train = balance_dataset(X_train, y_train, labels)

# Now you can use X_balanced and y_balanced for training

# 5. Vectorization

## 5.1 Raw Count

In [ ]:
# Initialization
max_features = 50
#max_features = 50
count_vectorizer = CountVectorizer(ngram_range=(1,3), min_df = 2, max_features=max_features)
# Fit and transform the text data
X_train = count_vectorizer.fit_transform(X_train)
X_test = count_vectorizer.transform(X_test)

print(X_train)
print("Size of the data set: ", X_train.shape)
print(X_test)
print("Size of the data set: ", X_test.shape)

## 5.2 TF-IDF 

In [17]:
# Initialization
max_features= None
#max_features=50
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,3), min_df = 2, max_features=max_features)
# Fit and transform the text data
X_train = tfidf_vectorizer.fit_transform(X_train)
X_test = tfidf_vectorizer.transform(X_test)
print(X_train)
print("Size of the data set: ", X_train.shape)
print(X_test)
print("Size of the data set: ", X_test.shape)

  (0, 79513)	0.034862310095075066
  (0, 132698)	0.03370561042656726
  (0, 66881)	0.034862310095075066
  (0, 54810)	0.034862310095075066
  (0, 100520)	0.03207533380855239
  (0, 54056)	0.03145553140398535
  (0, 64793)	0.028966525255446726
  (0, 46598)	0.034862310095075066
  (0, 4535)	0.023885526810942893
  (0, 16525)	0.034862310095075066
  (0, 42064)	0.034862310095075066
  (0, 120542)	0.03280840446015075
  (0, 153137)	0.034862310095075066
  (0, 80881)	0.025740485165814224
  (0, 72099)	0.03207533380855239
  (0, 20377)	0.034862310095075066
  (0, 18392)	0.03280840446015075
  (0, 72045)	0.03280840446015075
  (0, 154327)	0.034862310095075066
  (0, 12910)	0.03280840446015075
  (0, 51196)	0.034862310095075066
  (0, 86426)	0.03370561042656726
  (0, 14249)	0.034862310095075066
  (0, 72040)	0.034862310095075066
  (0, 95361)	0.034862310095075066
  :	:
  (6431, 19865)	0.032601385243252456
  (6431, 43431)	0.022866874258593753
  (6431, 83419)	0.013764555492864356
  (6431, 37435)	0.011911782458839372
 

### 5.2.1 TF-IDF with "!"

In [ ]:
# Use TfidfVectorizer with a custom token pattern to include punctuation
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r'(?u)\b\w+\b|[!]')

# Fit and transform the text data
X_train = tfidf_vectorizer.fit_transform(X_train)
X_test = tfidf_vectorizer.transform(X_test)
print(X_train)
print("Size of the data set: ", X_train.shape)
print(X_test)
print("Size of the data set: ", X_test.shape)

### 5.2.2 TF-IDF if you imported the data in 2.2 (sentiment analysis)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import hstack  # To combine sparse matrices

# Initialize TF-IDF Vectorizer
max_features = None
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=2, max_features=max_features)

# Fit and transform the preprocessed plot for the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['preprocessed_plot'])

# Transform the preprocessed plot for the test data
X_test_tfidf = tfidf_vectorizer.transform(X_test['preprocessed_plot'])

# Convert the sentiment column into a 2D array (needed to concatenate with sparse matrices)
X_train_sentiment = np.array(X_train['sentiment']).reshape(-1, 1)
X_test_sentiment = np.array(X_test['sentiment']).reshape(-1, 1)

# Combine the TF-IDF features and sentiment for the training and test sets
X_train_combined = hstack([X_train_tfidf, X_train_sentiment])
X_test_combined = hstack([X_test_tfidf, X_test_sentiment])

# Now you can use X_train_combined and X_test_combined for model training
print(X_train_combined)
print("Size of the training set: ", X_train_combined.shape)
print(X_test_combined)
print("Size of the test set: ", X_test_combined.shape)

# 6. Classifiers

## 6.1 Naive Bayes

In [ ]:
print("==== MultinomialNB ====")
clf = MultinomialNB()

clf.fit(X_train, y_train)
x_pred=clf.predict(X_test)
print(classification_report(y_pred=x_pred, y_true=y_test, labels = labels, zero_division=1.))

## 6.2 Suport Vector Machine

### With detailed evaluation metrics

In [ ]:
# === Train SVM model ===
print("==== SVM ====")
clf = svm.SVC(kernel='linear')

# Fit the model
clf.fit(X_train, y_train)

# Predict using the test set
x_pred = clf.predict(X_test)

# List of genres (labels)
labels = np.unique(y_test)

# === Classification Report ===
# Print overall accuracy, precision, recall, and f1-score by genre
print("\nClassification Report:")
report = classification_report(y_pred=x_pred, y_true=y_test, labels=labels, zero_division=1, output_dict=True)
report_df = pd.DataFrame(report).transpose()
print(report_df)

# === Overall Model Metrics ===
overall_accuracy = accuracy_score(y_test, x_pred)
overall_precision = report_df.loc['weighted avg']['precision']
overall_recall = report_df.loc['weighted avg']['recall']
overall_f1 = report_df.loc['weighted avg']['f1-score']

print("\nOverall Model Performance:")
print(f"Accuracy: {overall_accuracy:.2%}")
print(f"Precision (weighted avg): {overall_precision:.2%}")
print(f"Recall (weighted avg): {overall_recall:.2%}")
print(f"F1-score (weighted avg): {overall_f1:.2%}")


# === Confusion Matrix ===
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_test, x_pred, labels=labels)
print(pd.DataFrame(conf_matrix, index=labels, columns=labels))

# Assuming y_test and x_pred are defined
if isinstance(y_test, pd.Series):
    y_test_array = y_test.to_numpy()  # Convert to NumPy array if it's a Series
else:
    y_test_array = y_test  # If it's already a NumPy array

# Calculate accuracy for each genre
for genre in labels:
    genre_idx = np.where(y_test_array == genre)[0]  # Indices for this genre in y_test
    genre_accuracy = accuracy_score(y_test_array[genre_idx], x_pred[genre_idx])  # Calculate accuracy for this genre
    print(f"{genre}: {genre_accuracy:.2%}")

disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=clf.classes_)
disp.plot()
disp.ax_.set_xticklabels(disp.ax_.get_xticklabels(), rotation=90, ha='right')
plt.show()

==== SVM ====


### Check examples where the model fails

In [ ]:
# === Example of Misclassified Plot ===
misclassified_indices = np.where(y_test != x_pred)[0]

if len(misclassified_indices) > 0:
    # Get the first misclassified example
    first_misclassified_index = misclassified_indices[70]
    misclassified_plot = x_test_raw[first_misclassified_index]  # Assuming X_test_raw contains the raw plot texts
    true_genre = y_test.iloc[first_misclassified_index]
    predicted_genre = x_pred[first_misclassified_index]

    print("\nExample of Misclassification:")
    print(f"Plot: {misclassified_plot}")
    print(f"Predicted Genre: {predicted_genre}")
    print(f"True Genre: {true_genre}")
else:
    print("\nNo misclassifications found.")

## 6.3 K-Nearest Neighbours

In [ ]:
print("==== KNN ====")
clf = KNeighborsClassifier()  

clf.fit(X_train, y_train)
x_pred=clf.predict(X_test)
print(classification_report(y_pred=x_pred, y_true=y_test, labels = labels, zero_division=1.))

## 6.4 Decision Tree

In [ ]:
print("==== DT ====")
clf = DecisionTreeClassifier()  

clf.fit(X_train, y_train)
x_pred=clf.predict(X_test)
print(classification_report(y_pred=x_pred, y_true=y_test, labels = labels, zero_division=1.))

# 7. Hyperparameter Tuning (SVM model)

In [ ]:
# Define the SVM model
clf = svm.SVC(kernel='linear')

# Define the parameter grid to sample from
param_dist = {
    'C': np.logspace(-3, 3, 7),  # Regularization parameter
    'gamma': ['scale', 'auto'] + list(np.logspace(-3, 3, 7)),  # Kernel coefficient
    'class_weight': [None, 'balanced'],  # Class weight
}

# Set up the random search with cross-validation
random_search = RandomizedSearchCV(
    clf, 
    param_distributions=param_dist,
    n_iter=50,  # Number of parameter settings to try
    scoring='accuracy',  # Metric for evaluation
    cv=5,  # Number of cross-validation folds
    verbose=1,  # Verbosity level
    n_jobs=-1,  # Use all available cores
    random_state=42  # For reproducibility
)

# Fit the random search model
random_search.fit(X_train, y_train)

# Get the best estimator
best_model = random_search.best_estimator_

# Predict using the best model
x_pred = best_model.predict(X_test)

# Print classification report
print("==== SVM with Random Search ====")
print(classification_report(y_true=y_test, y_pred=x_pred, labels=labels, zero_division=1))

# Print the best hyperparameters
print("Best hyperparameters found:")
print(random_search.best_params_)